In [1]:
import sys
print(sys.executable)

/usr/bin/python3.6


In [2]:
import sys
import matplotlib.pyplot as plt
from Strats import Lazy, Naive, Conserv_Lazy, Conserv

In [3]:
class Evaluate:
    pw = [100000000, 1000000, 10000, 100]
    maxVal = 0
    maxArg = []
    def __init__(self, cls, data):
        self.cls = cls
        self.data = data
    def ReturnRate(self, args):
        ret = self.cls(self.data, args)
        self.cls.reset()
        print("with:", args, ret)
        return ret
    def loopOver(self, Layer, args):
        if(Layer >= self.cls.arg_count):
            tmp = self.ReturnRate(args)
            if tmp > self.maxVal:
                self.maxVal = tmp
                self.maxArg = args[:]
            return
        for i in range(self.cls.minArg[Layer], self.cls.MaxArg[Layer]):
            #print("Layer", Layer)
            args[Layer] = i
            self.loopOver(Layer+1, args)
    def FindOptArgs(self):
        self.loopOver(0, [0]*self.cls.arg_count)
        return self.maxVal, self.maxArg
    
    def name(self):
        return self.cls.name

In [4]:
# Process input
#DSI, IAU, LQD, SPY
files = ['DSI.csv', 'IAU.csv', 'LQD.csv', 'SPY.csv']
for file in files:
    f = open(file, 'r')
    closeP = []
    i = 0
    f.readline()
    while(True):
        line = f.readline()
        if not line:
            break
        closeP.append(float(line.split(',')[5]))
    s = Conserv()
    e = Evaluate(s, closeP)
    print(e.name(), e.FindOptArgs(), file)
    #plt.plot(closeP, label=file)
    #plt.show()

with: [3] 17929.580702000007
with: [4] 21671.592202999993
with: [5] 22870.14589699999
with: [6] 20979.918485
with: [7] 22766.336822999998
with: [8] 26513.642382
with: [9] 26513.642382
Conserv (26513.642382, [8]) DSI.csv
with: [3] 14697.871218999997
with: [4] 23047.186999999984
with: [5] 25356.597999999998
with: [6] 18339.166
with: [7] 18386.941999999995
with: [8] 29970.681999999997
with: [9] 33024.202
Conserv (33024.202, [9]) IAU.csv
with: [3] 17209.014568999995
with: [4] 16092.568816000008
with: [5] 19075.608160000007
with: [6] 13883.765967999998
with: [7] 13648.577164000002
with: [8] 12974.707896999998
with: [9] 12738.143685000001
Conserv (19075.608160000007, [5]) LQD.csv
with: [3] 16891.644397000004
with: [4] 17235.78595899999
with: [5] 17154.041326000006
with: [6] 24181.442921999995
with: [7] 32172.998273000005
with: [8] 32960.78845
with: [9] 32960.78845
Conserv (32960.78845, [8]) SPY.csv
